In [1]:
import pandas as pd
import math


content = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

import jieba
def cut(string):
    return list(jieba.cut(string))

content = content.fillna('')
all_news = content['content']

from tqdm import tqdm_notebook
all_occurences = []
for c in tqdm_notebook(all_news, total = len(all_news)):
    all_occurences.append(set(cut(c)))
    
def TF(term, cutted_passage):
    all_count = len(cutted_passage)
    return cutted_passage.count(term)/all_count

D = len(all_occurences)

def IDF(term):
    term_count = 0
    for passage_terms in all_occurences:
        if term in passage_terms:
            term_count += 1
    eps = 1e-6
    term_count += eps
    return math.log10(D/term_count)

def TF_IDF(term, cutted_passage):
    return TF(term, cutted_passage) * IDF(term)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Mark\AppData\Local\Temp\jieba.cache
Loading model cost 1.002 seconds.
Prefix dict has been built succesfully.


In [ ]:
import re

corpus_list = []
for index, row in tqdm_notebook(content.iterrows(), total = content.count()['content']):
    corpus_list += cut(''.join(re.findall(r'[\w\d]+', row['content'])))
    
with open('all_cut_words.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(corpus_list))

In [1]:
from gensim.models import Word2Vec as w2v
from gensim.models.word2vec import LineSentence
news_w2v_model = w2v(sentences=LineSentence(source='all_cut_words.txt'))

D:\ProgramFiles\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
from collections import defaultdict


def get_related_words(initial_words, model):
    max_size = 500
    seen = defaultdict(int)
    unseen = [initial_words]
    
    while unseen and len(seen) < max_size:
        node = unseen.pop(0)

        new_expanding = [w for w, s in model.most_similar(node, topn=20)]
        
        unseen += new_expanding
        
        seen[node] += 1  
    
    del seen[initial_words]
    return seen

In [22]:
sorted(get_related_words('说',news_w2v_model).items(), key=lambda kv: kv[1], reverse=True)

D:\ProgramFiles\Anaconda\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


[('表示', 61),
 ('指出', 61),
 ('认为', 55),
 ('坦言', 51),
 ('告诉', 43),
 ('看来', 40),
 ('透露', 40),
 ('介绍', 33),
 ('明说', 31),
 ('中说', 31),
 ('强调', 30),
 ('所说', 30),
 ('提到', 29),
 ('说道', 28),
 ('称', 26),
 ('文说', 25),
 ('声称', 22),
 ('建议', 19),
 ('呼吁', 18),
 ('提及', 18),
 ('地说', 15),
 ('直言', 15),
 ('普遍认为', 13),
 ('时说', 12),
 ('批评', 12),
 ('重申', 11),
 ('提出', 11),
 ('明确指出', 11),
 ('觉得', 11),
 ('宣称', 11),
 ('猜测', 11),
 ('特别强调', 10),
 ('写道', 10),
 ('引用', 10),
 ('相信', 9),
 ('说法', 9),
 ('暗示', 9),
 ('证实', 9),
 ('解释', 9),
 ('谈到', 8),
 ('深知', 8),
 ('问过', 8),
 ('称赞', 8),
 ('感慨', 8),
 ('承认', 8),
 ('部副', 8),
 ('常务', 8),
 ('敦促', 8),
 ('主张', 8),
 ('回答', 8),
 ('纳说', 7),
 ('问', 7),
 ('反问', 7),
 ('还称', 7),
 ('中称', 7),
 ('指责', 7),
 ('干涉', 7),
 ('观点', 6),
 ('看好', 6),
 ('余刚', 6),
 ('董', 6),
 ('披露', 6),
 ('努钦称', 6),
 ('明确提出', 6),
 ('描述', 6),
 ('提醒', 6),
 ('说完', 5),
 ('担心', 5),
 ('留意到', 5),
 ('见到', 5),
 ('看到', 5),
 ('问道', 5),
 ('眼中', 5),
 ('事实上', 5),
 ('而言', 5),
 ('坦承', 5),
 ('深有体会', 5),
 ('了解', 5),
 ('估算', 5),
 ('获悉', 

In [24]:
news_w2v_model.most_similar('说', topn=80)

D:\ProgramFiles\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('表示', 0.8433493971824646),
 ('认为', 0.7726807594299316),
 ('指出', 0.7580080628395081),
 ('告诉', 0.7162491083145142),
 ('看来', 0.7085323333740234),
 ('坦言', 0.7033018469810486),
 ('称', 0.6771471500396729),
 ('文说', 0.6369844675064087),
 ('明说', 0.634353518486023),
 ('介绍', 0.6171056032180786),
 ('透露', 0.6081358194351196),
 ('中说', 0.6044511795043945),
 ('强调', 0.5948407649993896),
 ('所说', 0.5840098857879639),
 ('说完', 0.5380305051803589),
 ('纳说', 0.5287141799926758),
 ('提到', 0.5230331420898438),
 ('时说', 0.5216999053955078),
 ('地说', 0.502944827079773),
 ('说道', 0.4897502362728119),
 ('写道', 0.48182275891304016),
 ('问', 0.48039889335632324),
 ('特别强调', 0.4793276786804199),
 ('建议', 0.4737555980682373),
 ('称赞', 0.4726901054382324),
 ('的话', 0.465386301279068),
 ('深有体会', 0.460712194442749),
 ('直言', 0.4534398913383484),
 ('相信', 0.4427821636199951),
 ('声称', 0.4423436224460602),
 ('尼格', 0.4415667653083801),
 ('及其', 0.4364781975746155),
 ('请问', 0.43345755338668823),
 ('呼吁', 0.4311087429523468),
 ('托说', 0.428